Bag-of-words representation
---

Goal: Represent texts as a vector of numbers for our ML tasks!

In [ ]:
# We cannot use X: ML models work with numerical features!
X = [
    "Scikit-learn makes ML easy, easy as 123", 
    "Learning TensorFlow for deep learning"
]

Idea: Create a feature for each word with the number of times the word appears as its value

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Scikit-learn provides a vectorizer transformer for that
vect = CountVectorizer()
X_encoded = vect.fit_transform(X);

In [ ]:
# Print vocabulary (the features) with their column index in the feature matrix
for word, index in vect.vocabulary_.items():
    print('"{}" with index {}'.format(word, index))

In [ ]:
# Input size
X_encoded.shape

Note: In practice, this approach usually leads to a large number of features (words) and only a few non-zero values per entry. For this reason, Scikit-learn stores the data as a "sparse matrix" that only stores non-zero values which is more memory efficient

In [ ]:
# Scikit-learn uses sparse matrices instead of Numpy arrays
X_encoded

But we can always get back the data as Numpy array

In [ ]:
X_encoded.toarray()

Notes

* `X_encoded[0, 3]` is 2 because "easy" (with index 3) appears twice in the first entry
* `X_encoded[1, 6]` is 2 because "learning" with index 6 appears twice in the second entry

Concrete use case - Sentiment analysis
---

Task: Classify movie reviews as being `positive` or `negative` about their movie. This is a binary classification task with text input.

Download the `Large Movie Review Dataset v1.0 ` data from https://ai.stanford.edu/~amaas/data/sentiment/ and extract it in a `aclImdb` folder next to this notebook

In [ ]:
from sklearn.datasets import load_files
import os

# Train set
train_data = load_files(os.path.join('aclImdb', 'train'), categories=['pos', 'neg'])

In [ ]:
# First review
train_data.data[0]

In [ ]:
# Classes
train_data.target_names

In [ ]:
train_data.target[0] # "positive"

In [ ]:
# Create X, y arrays
vect = CountVectorizer()
y = train_data.target
X = train_data.data
X_encoded = vect.fit_transform(X)

# Vocabulary and input
print('Training size:', len(train_data.data))
print('Vocabulary size:', len(vect.vocabulary_))
print('Input shape:', X_encoded.shape)
print('Target shape:', y.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Use logistic regression to classify reviews
grid_cv = GridSearchCV(
    estimator=LogisticRegression(solver='liblinear'),
    param_grid={
        'C': [0.1, 1, 10]
    },
    cv=3,
    return_train_score=True
)
grid_cv.fit(X_encoded, y);

In [ ]:
import pandas as pd

# Collect results in a DataFrame
cv_results = pd.DataFrame(grid_cv.cv_results_)
cols = ['param_C', 'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']
cv_results[cols].sort_values('mean_test_score', ascending=False)

Variants
---

There are also slightly more sophisticated methods like TD-IDF which normalizes counts by popularity of the word - the idea is that words that appear in many entries are less relevant to solve our task. A common example are stopwords like "the", "is", "a" but common words might also be specific to our data set ex. "movie", "actor" are common words in our case and irrelevant to classify the review as positive or negative.

In addition to logistic regressions, the Naive Bayes classifier is another baseline to evaluate when working with texts

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Create a pipeline this time
pipe = Pipeline([
    ('vect', None),
    ('clf', None)
])

# Try with simple counts and TF-IDF / Logistic regressions and Naive Bayes
grid_cv = GridSearchCV(
    estimator=pipe,
    param_grid={
        'vect': [CountVectorizer(), TfidfVectorizer()],
        'clf': [LogisticRegression(solver='liblinear'), MultinomialNB()]
    },
    cv=3,
    return_train_score=True,
    scoring='accuracy'
)
grid_cv.fit(X, y)

# Collect results in a DataFrame
cv_results = pd.DataFrame({
    'clf': [type(clf).__name__ for clf in grid_cv.cv_results_['param_clf']],
    'vect': [type(clf).__name__ for clf in grid_cv.cv_results_['param_vect']],
    'mean_test_score': grid_cv.cv_results_['mean_test_score'],
    'std_test_score': grid_cv.cv_results_['std_test_score'],
    'mean_train_score': grid_cv.cv_results_['mean_train_score'],
    'std_train_score': grid_cv.cv_results_['std_train_score']
})
cv_results.sort_values('mean_test_score', ascending=False)

Final test evaluation
---

In [ ]:
test_data = load_files(os.path.join('aclImdb', 'test'), categories=['pos', 'neg'])
grid_cv.score(test_data.data, test_data.target) # Score best estimator

Additional ressources
---

Scikit-learn provides many options for vectorizers, make sure to check the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) documentations